In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.10.1


In [2]:
# Get the current working directory
# cwd = os.getcwd()

X, y = [], []
path = "wheat_leaf"
class_dict = {'Healthy':0, 
              'septoria':1, 
              'stripe_rust':2}

for each_class in class_dict.keys():
    # Get all the files in the path directory
    files = os.listdir(path + '/' + each_class)
    # print(files)
    for each_file in files:
        if each_file[0] == '.':
            continue
            
#         image = tf.keras.utils.load_img(path + '/' + each_class + '/' + each_file, 
#                                         target_size=(224, 224))
#         image_arr = tf.keras.utils.img_to_array(image)
        img = cv.imread(path + '/' + each_class + '/' + each_file)
        image_arr = cv.resize(img, (224,224))
        X.append(image_arr)
        y.append(class_dict[each_class])
    print('successfully loaded for class:',  each_class)

successfully loaded for class: Healthy
successfully loaded for class: septoria
successfully loaded for class: stripe_rust


In [3]:
X, y = np.array(X), np.array(y)
X.shape, y.shape

((407, 224, 224, 3), (407,))

In [31]:
from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

X_train, X_dummy, y_train, y_dummy = train_test_split(X, y, train_size=0.9, shuffle=True, random_state=123, stratify=y)
print(f'training',X_train.shape, X_dummy.shape, y_train.shape, y_dummy.shape)

X_val, X_test, y_val, y_test = train_test_split(X_dummy, y_dummy, train_size=0.5, shuffle=True, random_state=123, stratify=y_dummy)
print(f'val - test',X_val.shape, X_test.shape, y_val.shape, y_test.shape)


training (366, 224, 224, 3) (41, 224, 224, 3) (366,) (41,)
val - test (20, 224, 224, 3) (21, 224, 224, 3) (20,) (21,)


In [32]:
X_train = X_train / 255
X_test = X_test / 255

In [33]:
# y_train.value_counts()
unique_values, counts = np.unique(y_train, return_counts=True)
unique_values, counts, (counts / len(y_train) * 100).round(2)

(array([0, 1, 2]),
 array([ 92,  87, 187], dtype=int64),
 array([25.14, 23.77, 51.09]))

In [34]:
# y_test.value_counts()
unique_values, counts = np.unique(y_test, return_counts=True)
unique_values, counts, (counts / len(y_test) * 100).round(2)

(array([0, 1, 2]),
 array([ 5,  5, 11], dtype=int64),
 array([23.81, 23.81, 52.38]))

In [11]:
# Convert labels to one-hot encoding
num_classes = 3
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_train.shape, y_test.shape

((366, 3), (41, 3))

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [13]:
# Define input image dimensions
img_width, img_height = 224, 224
num_classes = 3  # Example number of classes

In [14]:
# Load the MobileNetV2 model pre-trained on ImageNet data
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the convolutional base
base_model.trainable = False

In [15]:
# Add custom dense layers on top of the convolutional base
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [16]:
# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

In [18]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [19]:
# Create data generators for training and validation with data augmentation
# train_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

# Define data augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=20,      # Random rotation within the range [-20, 20] degrees
    width_shift_range=0.1,  # Random horizontal shift by up to 10% of the image width
    height_shift_range=0.1, # Random vertical shift by up to 10% of the image height
    zoom_range=0.2,         # Random zoom by up to 20%
    horizontal_flip=True,   # Random horizontal flipping
    fill_mode='nearest'
)

In [20]:
# Train the model
history = model.fit(train_datagen.flow(X_train, y_train, batch_size=64),
                    epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
6/6 [==============================] - 8s 1s/step - loss: 0.9533 - accuracy: 0.5546 - val_loss: 0.6615 - val_accuracy: 0.6829
Epoch 2/10
6/6 [==============================] - 8s 1s/step - loss: 0.3956 - accuracy: 0.8197 - val_loss: 0.2928 - val_accuracy: 0.9268
Epoch 3/10
6/6 [==============================] - 6s 958ms/step - loss: 0.2770 - accuracy: 0.8825 - val_loss: 0.2820 - val_accuracy: 0.8780
Epoch 4/10
6/6 [==============================] - 8s 1s/step - loss: 0.1701 - accuracy: 0.9454 - val_loss: 0.1704 - val_accuracy: 0.9268
Epoch 5/10
6/6 [==============================] - 7s 1s/step - loss: 0.1326 - accuracy: 0.9617 - val_loss: 0.1417 - val_accuracy: 0.9756
Epoch 6/10
6/6 [==============================] - 7s 1s/step - loss: 0.1073 - accuracy: 0.9727 - val_loss: 0.1499 - val_accuracy: 0.9756
Epoch 7/10
6/6 [==============================] - 8s 1s/step - loss: 0.0934 - accuracy: 0.9672 - val_loss: 0.1265 - val_accuracy: 0.9756
Epoch 8/10
6/6 [======================

In [21]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

2/2 [==============================] - 0s 111ms/step - loss: 0.1302 - accuracy: 0.9756
Test loss: 0.13023006916046143
Test accuracy: 0.9756097793579102


In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [23]:
def MobileNetV2(input_shape, num_classes):
    """Creates a custom MobileNetV2 model."""
    inputs = tf.keras.Input(shape=input_shape)

    # Entry flow
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = inverted_residual_block(x, filters=16, strides=1, expansion=1)
    x = inverted_residual_block(x, filters=24, strides=2, expansion=6)
    x = inverted_residual_block(x, filters=24, strides=1, expansion=6)
    x = inverted_residual_block(x, filters=32, strides=2, expansion=6)
    x = inverted_residual_block(x, filters=32, strides=1, expansion=6)
    x = inverted_residual_block(x, filters=32, strides=1, expansion=6)

    x = inverted_residual_block(x, filters=64, strides=2, expansion=6)
    x = inverted_residual_block(x, filters=64, strides=1, expansion=6)
    x = inverted_residual_block(x, filters=64, strides=1, expansion=6)
    x = inverted_residual_block(x, filters=96, strides=1, expansion=6)
    x = inverted_residual_block(x, filters=96, strides=1, expansion=6)
    x = inverted_residual_block(x, filters=96, strides=1, expansion=6)

    x = inverted_residual_block(x, filters=160, strides=2, expansion=6)
    x = inverted_residual_block(x, filters=160, strides=1, expansion=6)
    x = inverted_residual_block(x, filters=160, strides=1, expansion=6)

    x = inverted_residual_block(x, filters=320, strides=1, expansion=6)

    # Exit flow
    x = layers.Conv2D(1280, (1, 1), strides=(1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x, name='mobilenet_v2')
    return model

def inverted_residual_block(inputs, filters, strides, expansion):
    """Creates an inverted residual block."""
    channel_axis = 1 if tf.keras.backend.image_data_format() == 'channels_first' else -1
    in_channels = inputs.get_shape().as_list()[channel_axis]
    x = inputs

    # Expansion phase
    if expansion != 1:
        x = layers.Conv2D(expansion * in_channels, kernel_size=1, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

    # Depthwise convolution
    x = layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Pointwise convolution
    x = layers.Conv2D(filters, kernel_size=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add shortcut connection if input and output shapes are the same and strides == 1
    if strides == 1 and in_channels == filters:
        x = layers.Add()([inputs, x])

    return x

In [24]:
# Define input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape for image classification
num_classes = 3  # Example number of classes

# Create the MobileNetV2 model
model = MobileNetV2(input_shape, num_classes)

# Print the model summary
model.summary()

Model: "mobilenet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 32  896         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                      

In [25]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
# Train the model
history1 = model.fit(train_datagen.flow(X_train, y_train, batch_size=64),
                    epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
6/6 [==============================] - 22s 4s/step - loss: 0.3965 - accuracy: 0.8224 - val_loss: 1.0789 - val_accuracy: 0.5122
Epoch 2/5
6/6 [==============================] - 27s 5s/step - loss: 0.3364 - accuracy: 0.8470 - val_loss: 1.0879 - val_accuracy: 0.2439
Epoch 3/5
6/6 [==============================] - 41s 7s/step - loss: 0.3337 - accuracy: 0.8552 - val_loss: 1.0493 - val_accuracy: 0.5122
Epoch 4/5
6/6 [==============================] - 40s 7s/step - loss: 0.2966 - accuracy: 0.8770 - val_loss: 1.0442 - val_accuracy: 0.5122
Epoch 5/5
6/6 [==============================] - 33s 5s/step - loss: 0.1714 - accuracy: 0.9372 - val_loss: 1.0520 - val_accuracy: 0.5122
